In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.nn.functional as F
import time

from utils import *
from model import LogisticRegression
from prepare_data import *

In [2]:
def My_loss(labels, logits, samples_per_cls, no_of_classes, loss_type, beta, gamma): 
    
    if loss_type == "CSL":
        assert beta == 1,"Cost_sensitive should set beta=1"
        
        prob_per_cls = np.array(samples_per_cls)/np.sum(np.array(samples_per_cls)) 
        weights = 1 / prob_per_cls
        weights = torch.tensor(weights / np.sum(weights) * no_of_classes).float()
        
        loss = F.cross_entropy(input = logits, target = labels, weight = weights)
    
    elif loss_type == "CBL":   # CB_loss
        assert beta != 0,"class balanced loss shouldn't set beta=0"
        
        effective_num = 1.0 - np.power(beta, samples_per_cls)
        weights = (1.0 - beta) / np.array(effective_num)
        weights = torch.tensor(weights / np.sum(weights) * no_of_classes).float()
        
        loss = F.cross_entropy(input = logits, target = labels, weight = weights)
    
    elif loss_type == "FL":   # CB_loss
        assert beta == 0,"focal loss should set beta=0"
        
        
        labels_one_hot = F.one_hot(labels, no_of_classes).float()
        
        alpha = 0.25
        weights = alpha * torch.ones(labels.shape[0], no_of_classes)
        BCLoss = F.binary_cross_entropy_with_logits(input = logits, target = labels_one_hot,reduction = "none")
        modulator = torch.exp(-gamma * labels_one_hot * logits - gamma * torch.log(1 + torch.exp(-1.0 * logits)))
        floss = modulator * BCLoss
        weighted_loss = weights * floss
        loss = torch.sum(weighted_loss)
        loss /= torch.sum(labels_one_hot)
    elif loss_type == "RDS":
        loss = F.cross_entropy(input = logits, target = labels)
        
    return loss

In [3]:
## Loss type selection

# loss_type = "CSL"
# loss_type = "FL"
# loss_type = "CBL"
loss_type = "RDS"
params = Params(loss_type)

## Load data
traindata = np.load(f'/mnt/multi-class_simu/real data/Audi/data/512_traindata_8cls.npz')
testdata = np.load(f'/mnt/multi-class_simu/real data/Audi/data/512_testdata_8cls.npz')

samples_per_cls,dataloader,Ns0,X1_torch,Y1_torch,X1,Y1 = mkdata(loss_type,traindata,testdata,params)

/mnt/icml_2025_multi-class/real data/code_submit/utils.py:125: RuntimeWarning: invalid value encountered in true_divide
  X_std = (X_non-Xmin)/(Xmax-Xmin); X_std[:,idx] = 0


In [4]:
# Train
model = LogisticRegression(params["input_size"], params["num_classes"])
optimizer = optim.SGD(model.parameters(), lr=params["lr"])
t1 = time.time()

for epoch in range(params["num_epochs"]):
    running_loss = 0.0
    if epoch == params["warm_up1"]:
        for param_group in optimizer.param_groups:
            param_group['lr'] *= params["multi_lr"]
    if epoch == params["warm_up2"]:
        for param_group in optimizer.param_groups:
            param_group['lr'] *= params["multi_lr"]
    for i, (inputs, labels) in enumerate(dataloader):
        outputs = model(inputs)
        loss = My_loss(labels, outputs, samples_per_cls, params["num_classes"],loss_type, params["beta"], params["gamma"])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch + 1}/{params["num_epochs"]}, Loss: {running_loss / len(dataloader)}')
t2 = time.time(); print(f'End: {np.round(t2-t1,4)}s')

Epoch 1/90, Loss: 0.8287117248773574
Epoch 2/90, Loss: 0.5741238868236542
Epoch 3/90, Loss: 0.5007968431711197
Epoch 4/90, Loss: 0.45617625415325164
Epoch 5/90, Loss: 0.42292893171310425
Epoch 6/90, Loss: 0.3984169238805771
Epoch 7/90, Loss: 0.3783349734544754
Epoch 8/90, Loss: 0.36161125004291533
Epoch 9/90, Loss: 0.34646953642368317
Epoch 10/90, Loss: 0.3339306643605232
Epoch 11/90, Loss: 0.32292487531900405
Epoch 12/90, Loss: 0.3124754500389099
Epoch 13/90, Loss: 0.30334572821855543
Epoch 14/90, Loss: 0.2951361933350563
Epoch 15/90, Loss: 0.2875497755408287
Epoch 16/90, Loss: 0.28055752605199813
Epoch 17/90, Loss: 0.27418875962495803
Epoch 18/90, Loss: 0.2684985277056694
Epoch 19/90, Loss: 0.2621381512284279
Epoch 20/90, Loss: 0.25747287154197696
Epoch 21/90, Loss: 0.2523703783750534
Epoch 22/90, Loss: 0.24774535447359086
Epoch 23/90, Loss: 0.24381056070327758
Epoch 24/90, Loss: 0.23900071829557418
Epoch 25/90, Loss: 0.23519689440727234
Epoch 26/90, Loss: 0.23171984404325485
Epoch 2

In [ ]:
# torch.save(model,'focal_loss.pth')

In [5]:
# Evaluation
model.eval()

outputs = model(X1_torch)
prob_hat = F.softmax(outputs, dim=1)
_, Yhat = torch.max(outputs.data, 1)
prob_hat = prob_hat.detach().numpy()
Yhat = Yhat.detach().numpy() 

w = model.linear.weight.detach().numpy() # [K,p+1]
b = model.linear.bias.detach().numpy() # [1,p+1]
w = np.hstack((b.reshape([-1,1]),w))
w_stand = w - w[0]
beta_hat = w_stand[1:].T 

test_accw(np.concatenate((np.ones([X1.shape[0],1]),X1),1), Y1, Ns0, beta_hat)

9 cls acc = 0.76,auc = 0.996
